In [5]:
import pandas as pd
import glob
import datetime
from datetime import date
import warnings
warnings.filterwarnings('ignore')
from functools import reduce
import re

In [605]:
exp = pd.read_csv(r'C:\NotBackedUp\Expense\CSV\DirDiv.csv',delimiter='|')

In [566]:
exp= exp.rename(columns={'DOMAIN/PORTFOLIO':'DOMAIN','TRIBE/TECH AREA':'TRIBE','JUNE':'AMOUNT'})

In [567]:
exp['DOMAIN'] = exp['DOMAIN'].replace(['Retail + Commercial Domain'],['Australia Retail and Commercial'])
exp['DOMAIN'] = exp['DOMAIN'].replace(['Enterprise Data + Intelligent Automation'],['Data Services'])
exp['DOMAIN'] = exp['DOMAIN'].replace(['New Zealand Domain'],['New Zealand'])
exp['DOMAIN'] = exp['DOMAIN'].replace(['Institutional Domain'],['Institutional'])
exp['DOMAIN'] = exp['DOMAIN'].replace(['Transformation - India (INR)'],['Technology Transformation'])
exp['DOMAIN'] = exp['DOMAIN'].replace(['Transformation - China (CNY)'],['Technology Transformation'])
exp['DOMAIN'] = exp['DOMAIN'].replace(['Transformation - Australia'],['Technology Transformation'])
exp['DOMAIN'] = exp['DOMAIN'].replace(['Transformation - Singapore (USD)'],['Technology Transformation'])
exp['TRIBE']= exp['TRIBE'].replace(['Enterprise Data Initiatives'],['Enterprise Data Platforms'])
exp['DOMAIN'] = exp['DOMAIN'].replace(['Technology HQ'],['Tech HQ - Major Programs'])
exp['TRIBE'] = exp['TRIBE'].replace(['Technology HQ'],['Tech HQ - Major Programs'])

In [2]:
basedomain = ['Australia Retail and Commercial', 'Banking Services', 'Business Automation and Integration', 'Data Services', 'Employee Experience', 'Infrastructure as a Service', 'Institutional', 'New Zealand', 'Security', 'Technology Business Management', 'Technology Operations', 'Technology Transformation', 'Wealth','CIO','Delivery Services','Tech HQ - Major Programs']

In [ ]:
exp = exp[exp.DOMAIN.isin(basedomain)]

In [569]:
exp[(exp['DOMAIN'] == 'Data Services')]

,EXPENSES,Division,DOMAIN,TRIBE,AMOUNT
47,Direct Expenses,Group Technology,Data Services,Business Automation,2350627.5
48,Direct Expenses,Group Technology,Data Services,Data General,332290.1
49,Direct Expenses,Group Technology,Data Services,Data Services - Customer,193754.6
50,Direct Expenses,Group Technology,Data Services,Data Services - Group,877509.2
51,Direct Expenses,Group Technology,Data Services,Enterprise Data Platforms,59560.4
52,Direct Expenses,Group Technology,Data Services,Data Ecosystem +Analytics,1098995.6
53,Direct Expenses,Group Technology,Data Services,Data Lifecycle Evolution,441813.6
54,Direct Expenses,Group Technology,Data Services,Enterprise Data Platforms,1793035.9
120,Divisionally Sponsored Projects,Group Technology,Data Services,Business Automation,-1600668.5
121,Divisionally Sponsored Projects,Group Technology,Data Services,Data General,-160609.6


In [570]:
Dir_exp = exp[exp['EXPENSES']=='Direct Expenses']
Dir_exp = Dir_exp[['Division','DOMAIN', 'TRIBE','AMOUNT']]
Dir_exp = Dir_exp.rename(columns={'Division':'DIVISION'})
Dir_exp = Dir_exp.groupby(['DIVISION','DOMAIN', 'TRIBE'])['AMOUNT'].sum().reset_index(name='DirectExpense')

In [571]:
Dir_exp[(Dir_exp['DOMAIN'] == 'Data Services')]

,DIVISION,DOMAIN,TRIBE,DirectExpense
13,Group Technology,Data Services,Business Automation,2350627.5
14,Group Technology,Data Services,Data Ecosystem +Analytics,1098995.6
15,Group Technology,Data Services,Data General,332290.1
16,Group Technology,Data Services,Data Lifecycle Evolution,441813.6
17,Group Technology,Data Services,Data Services - Customer,193754.6
18,Group Technology,Data Services,Data Services - Group,877509.2
19,Group Technology,Data Services,Enterprise Data Platforms,1852596.3


In [572]:
Dir_exp.loc[(Dir_exp['DOMAIN'] == 'Tech HQ - Major Programs')]

,DIVISION,DOMAIN,TRIBE,DirectExpense
56,Group Technology,Tech HQ - Major Programs,Tech HQ - Major Programs,6841888.3


In [573]:
Div_exp = exp[exp['EXPENSES']=='Divisionally Sponsored Projects']
Div_exp = Div_exp[['Division','DOMAIN', 'TRIBE','AMOUNT']]
Div_exp = Div_exp.rename(columns={'Division':'DIVISION'})
Div_exp = Div_exp.groupby(['DIVISION','DOMAIN', 'TRIBE'])['AMOUNT'].sum().reset_index(name='DivisionalExpense')
Div_exp.head()

,DIVISION,DOMAIN,TRIBE,DivisionalExpense
0,Group Technology,Australia Retail and Commercial,Business Owner,-1800542.6
1,Group Technology,Australia Retail and Commercial,Core System,-1710134.8
2,Group Technology,Australia Retail and Commercial,Digital,-9079802.2
3,Group Technology,Australia Retail and Commercial,Home Lending,-1054490.7
4,Group Technology,Australia Retail and Commercial,Retail + Commercial Domain HQ,-1213592.6


In [574]:
Div_exp[(Div_exp['DOMAIN'] == 'Data Services')]

,DIVISION,DOMAIN,TRIBE,DivisionalExpense
13,Group Technology,Data Services,Business Automation,-1600668.5
14,Group Technology,Data Services,Data Ecosystem +Analytics,-432190.8
15,Group Technology,Data Services,Data General,-160609.6
16,Group Technology,Data Services,Data Lifecycle Evolution,-243853.6
17,Group Technology,Data Services,Data Services - Customer,-150760.5
18,Group Technology,Data Services,Data Services - Group,-482879.0
19,Group Technology,Data Services,Enterprise Data Platforms,-279119.0


In [6]:
FTE = pd.read_csv(r'C:\NotBackedUp\Expense\squad.csv',delimiter='|')

In [7]:
list(FTE)

['Month',
 'Division',
 'LOB',
 'Sub-LOB',
 'Sub-LOB 1',
 'Employee Name',
 'Employment Status',
 'Job Code Description',
 'Job Family',
 'Job Function',
 'FTE']

In [8]:
FTE.Division.unique()

array(['Australia', 'Australia Property', 'Central Functions',
       'Corporate Centre', 'Enablement Functions',
       'Group Operations and Services', 'Group Technology',
       'Institutional', 'Invalid FTE', 'New Zealand', 'Pacific',
       'Procurement', 'Wealth Australia'], dtype=object)

In [9]:
FTE['Division'] = FTE['Division'].replace(['Australia'],['Aus R & C'])

In [10]:
FTE['LOB'] = FTE['LOB'].replace(['Retail + Commercial Domain'],['Australia Retail and Commercial'])
FTE['LOB'] = FTE['LOB'].replace(['Enterprise Data + Intelligent Automation'],['Data Services'])
FTE['LOB'] = FTE['LOB'].replace(['New Zealand Domain'],['New Zealand'])
FTE['LOB'] = FTE['LOB'].replace(['Institutional Domain'],['Institutional'])
FTE['LOB'] = FTE['LOB'].replace(['Transformation - India (INR)'],['Technology Transformation'])
FTE['LOB'] = FTE['LOB'].replace(['Transformation - China (CNY)'],['Technology Transformation'])
FTE['LOB'] = FTE['LOB'].replace(['Transformation - Australia'],['Technology Transformation'])
FTE['LOB'] = FTE['LOB'].replace(['Transformation - Singapore (USD)'],['Technology Transformation'])
FTE['Sub-LOB']= FTE['Sub-LOB'].replace(['Enterprise Data Initiatives'],['Enterprise Data Platforms'])
FTE['LOB'] = FTE['LOB'].replace(['Technology HQ'],['Tech HQ - Major Programs'])
FTE['Sub-LOB'] = FTE['Sub-LOB'].replace(['Technology HQ'],['Tech HQ - Major Programs'])
FTE = FTE[FTE.LOB.isin(basedomain)]

In [11]:
FTE = FTE.rename(columns={'Division':'DIVISION','LOB':'DOMAIN','Sub-LOB':'TRIBE'})

In [12]:
#FTE.loc[(FTE['DOMAIN'] == 'Employee Experience')&(FTE['TRIBE']== 'EE - Testing Services') & (FTE['Job Family']== 'Testing')]
FTE = FTE[['DIVISION','DOMAIN','TRIBE','Job Family']]

In [581]:
sal = pd.read_csv(r'C:\NotBackedUp\Expense\CSV\Sal.csv',delimiter='|')
#FTE.loc[(FTE['DOMAIN'] == 'Employee Experience')&(FTE['TRIBE']== 'EE - Testing Services') & (FTE['Job Family']== 'Testing')]

In [582]:
sal['DOMAIN'] = sal['DOMAIN'].replace(['Retail + Commercial Domain'],['Australia Retail and Commercial'])
sal['DOMAIN'] = sal['DOMAIN'].replace(['Enterprise Data + Intelligent Automation'],['Data Services'])
sal['DOMAIN'] = sal['DOMAIN'].replace(['New Zealand Domain'],['New Zealand'])
sal['DOMAIN'] = sal['DOMAIN'].replace(['Institutional Domain'],['Institutional'])
sal['DOMAIN'] = sal['DOMAIN'].replace(['Transformation - India (INR)'],['Technology Transformation'])
sal['DOMAIN'] = sal['DOMAIN'].replace(['Transformation - China (CNY)'],['Technology Transformation'])
sal['DOMAIN'] = sal['DOMAIN'].replace(['Transformation - Australia'],['Technology Transformation'])
sal['DOMAIN'] = sal['DOMAIN'].replace(['Transformation - Singapore (USD)'],['Technology Transformation'])
sal['TRIBE']= sal['TRIBE'].replace(['Enterprise Data Initiatives'],['Enterprise Data Platforms'])
sal['DOMAIN'] = sal['DOMAIN'].replace(['Technology HQ'],['Tech HQ - Major Programs'])
sal['TRIBE'] = sal['TRIBE'].replace(['Technology HQ'],['Tech HQ - Major Programs'])
sal = sal[sal.DOMAIN.isin(basedomain)]

In [583]:
#sal = sal.rename(columns={'Division':'DIVISION','Domain':'DOMAIN','Tribe':'TRIBE'})
sal = sal.groupby(['DIVISION','DOMAIN','TRIBE'])['SAL'].sum().reset_index(name='SalaryExpense')

In [584]:
sal.loc[(sal['DOMAIN'] == 'Data Services')]

,DIVISION,DOMAIN,TRIBE,SalaryExpense
13,Group Technology,Data Services,Business Automation,1272274.0
14,Group Technology,Data Services,Data Ecosystem +Analytics,581860.0
15,Group Technology,Data Services,Data General,175992.0
16,Group Technology,Data Services,Data Lifecycle Evolution,325869.0
17,Group Technology,Data Services,Data Services - Customer,584526.0
18,Group Technology,Data Services,Data Services - Group,647363.0
19,Group Technology,Data Services,Enterprise Data Platforms,189111.0


In [585]:
sal.head()

,DIVISION,DOMAIN,TRIBE,SalaryExpense
0,Group Technology,Australia Retail and Commercial,Business Owner,628659.0
1,Group Technology,Australia Retail and Commercial,Core System,1074888.0
2,Group Technology,Australia Retail and Commercial,Digital,3589607.0
3,Group Technology,Australia Retail and Commercial,Home Lending,525483.0
4,Group Technology,Australia Retail and Commercial,Retail + Commercial Domain HQ,455018.0


In [586]:
RBPC = FTE.groupby(['DIVISION','DOMAIN','TRIBE','Job Family'])['Job Family'].count().reset_index(name='ActiveStaffCount')

In [587]:
RBPC.head()

,DIVISION,DOMAIN,TRIBE,Job Family,ActiveStaffCount
0,Group Technology,Australia Retail and Commercial,Business Owner,Agile Management,1
1,Group Technology,Australia Retail and Commercial,Business Owner,Analytics & Reporting,1
2,Group Technology,Australia Retail and Commercial,Business Owner,Architecture,3
3,Group Technology,Australia Retail and Commercial,Business Owner,Data Intelligence,3
4,Group Technology,Australia Retail and Commercial,Business Owner,Project Business Analysis,1


In [588]:
#RBPC.loc[(RBPC['DOMAIN'] == 'Employee Experience')&(RBPC['TRIBE']== 'EE - Testing Services') & (RBPC['Job Family']== 'Testing')]

In [14]:
FTECount = FTE.groupby(['DIVISION','DOMAIN','TRIBE'])['Job Family'].count().reset_index(name='Total_FTE')
#FTECount.loc[(FTECount['DOMAIN'] == 'Employee Experience')&(FTECount['TRIBE']== 'EE - Testing Services')]

In [590]:
#FTECount.loc[(FTECount['DOMAIN'] == 'Employee Experience')&(FTECount['TRIBE']== 'EE - Testing Services')]

In [591]:
dfs = [Dir_exp,Div_exp,FTECount,sal]

In [592]:
df_final = reduce(lambda left,right: pd.merge(left,right,on=['DIVISION','DOMAIN','TRIBE']), dfs)

In [593]:
df_final['Month'] = 'Jun-20'

In [594]:
list(df_final)

['DIVISION',
 'DOMAIN',
 'TRIBE',
 'DirectExpense',
 'DivisionalExpense',
 'Total_FTE',
 'SalaryExpense',
 'Month']

In [595]:
df_final = df_final[['DIVISION','DOMAIN','TRIBE','Month','Total_FTE','DirectExpense','DivisionalExpense','SalaryExpense']]
df_final.head()

,DIVISION,DOMAIN,TRIBE,Month,Total_FTE,DirectExpense,DivisionalExpense,SalaryExpense
0,Group Technology,Australia Retail and Commercial,Business Owner,Jun-20,49,2086795.3,-1800542.6,628659.0
1,Group Technology,Australia Retail and Commercial,Core System,Jun-20,112,2671090.7,-1710134.8,1074888.0
2,Group Technology,Australia Retail and Commercial,Digital,Jun-20,369,11723806.8,-9079802.2,3589607.0
3,Group Technology,Australia Retail and Commercial,Home Lending,Jun-20,49,1661173.8,-1054490.7,525483.0
4,Group Technology,Australia Retail and Commercial,Retail + Commercial Domain HQ,Jun-20,23,1560000.4,-1213592.6,455018.0


In [596]:
df_final.to_csv(r'C:\NotBackedUp\CSV\FinalExp.csv',sep='|',index=None,header=None)

In [597]:
#RoleBased = pd.read_csv(r'C:\NotBackedUp\CSV\EXp\RBPC.csv',delimiter='|')

In [598]:
#RoleBased.head()

In [599]:
efs = [df_final,RBPC]

In [600]:
RBPC_Final = reduce(lambda left,right: pd.merge(left,right,on=['DIVISION','DOMAIN','TRIBE']), efs)

In [601]:
list(RBPC_Final)

['DIVISION',
 'DOMAIN',
 'TRIBE',
 'Month',
 'Total_FTE',
 'DirectExpense',
 'DivisionalExpense',
 'SalaryExpense',
 'Job Family',
 'ActiveStaffCount']

In [603]:
RBPC_Final.to_csv(r'C:\NotBackedUp\CSV\RBPCExp.csv',sep='|',index=None,header=None)

In [602]:
RBPC_Final = RBPC_Final[['DIVISION','DOMAIN','TRIBE','Job Family','ActiveStaffCount']]